# MNISTの学習とモデル・重みの保存

---

まず、[OpenHPC-v2テンプレート](https://github.com/nii-gakunin-cloud/ocs-templates/tree/master/OpenHPC-v2)のクローンを作成し、以下の手順で学習システムを構築してください。OpenHPC-v2のテンプレートは内部で複数のテンプレートに分かれています。実行順序はテンプレートの番号順ではありません。

1. 010-パラメータ設定.ipynb
1. 020-OpenHPCの起動.ipynb
1. 032-設定ファイルの編集-GRESの登録.ipynb
1. 071-DockerEngineのインストール.ipynb
1. 051-ユーザの追加.ipynb
1. 152-NGCのコンテナ実行-TensorFlow.ipynb

これらテンプレートの実行により、GPU ノードを使用する TensorFlow の環境が構築されます。この環境は、本ハンズオン以外の機械学習にも利用できます。

次に、本テンプレートを使用して手書き数字認識アプリである MNIST の学習を、構築した OpenHPC 環境で実行し、結果のモデル・重みを保存します。OpenHPCテンプレートの`152-NGCのコンテナ実行-TensorFlow.ipynb`にも MNIST の学習ステップがありますが、モデル・重みを保存していないため本テンプレートを用意しました。

学習が終わったら、`2.Recognition_system`で手書き認識を試しください。

OpenHPC環境を削除するには、OpenHPC テンプレートの`920-OpenHPC環境の削除.ipynb`を実行します。

もし、OpenHPC環境を後日再度利用する可能性がある場合、次の２つの方法があります。

* '920-OpenHPC環境の削除.ipynb'で環境を削除し、再び必要になった時に、上記ステップで再構築します。再構築には時間と手間を要しますが、削除から再構築までの間のコストはかかりません。
* 再利用する可能性がある場合は、'920-OpenHPC環境の削除.ipynb'で環境を削除せずに、'911-ノードの停止.ipynb'で環境を停止し、再び必要になった時に '912-ノードの再開.ipynb' で再起動します。迅速に環境を再現できますが、停止中も若干のコストが発生します。

## 実行パラメタの修正

必要に応じ、実行するためのパラメタを記入・修正してください。
* ohpc_path は OpenHPC 構築時に使用した作業ディレクトリのパスを指定してください。（例：~/ocs-templates/OpenHPC-v2/work）
* save_dir は、本ハンズオン教材を展開するディレクトリにより異なりますので、各自の環境に合わせて記入してください

In [ ]:
# slurmマスターノードのIPアドレス
master_address = '172.30.2.120'

# MNISTの学習を実行するユーザー名
user = 'user00'

# 上記ユーザーのSSH秘密鍵
ssh_identity = '~/.ssh/id_rsa'

# 学習済みモデル・重みファイルの保存先ディレクトリ
save_dir = '../2.Recognition_system/data'

## 実行環境の確認

実行時のパラメタを設定します。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
# print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
# print(ssh_opts)

# 使用するノード数
nodes = 1

計算ノードで Docker を利用できることを確認します。エラーが出ないことを確認してください。

In [ ]:
!ssh {ssh_opts} {target} 'srun -N {nodes} docker info'

上のセルがエラーになった場合は Docker Engineがインストールされていないか docker を実行する権限が与えられていません。

以下のようなエラーが表示された場合は Docker Engine がインストールされていません。OpenHPCテンプレート作業ディレクトリで「071-DockerEngineのインストール.ipynb」を実行してDocker Engineを利用できるようにしてください。
```
slurmstepd: error: execve(): docker: No such file or directory
```

また、以下のようなエラーが表示された場合は docker を実行する権限が与えられていません。OpenHPCテンプレート作業ディレクトリの「051-ユーザの追加.ipynb」の「グループの追加」などを参考に`docker`グループへの追加を行ってください。
```
ERROR: Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: 
```

SlurmのGeneric Resource(GRES)としてGPUが登録されていることを確認します。gpuが1以上になっていることを確認してください。

In [ ]:
!ssh {ssh_opts} {target} sinfo -N --Format=NodeHost,Gres | grep -w gpu

上のセルがエラーになった場合はSlurmにGPUがGRESとして登録されていません。OpenHPCテンプレート作業ディレクトリで「032-設定ファイルの編集-GRESの登録.ipynb」を実行してGRESの設定をしてください。

## OpenHPC 環境で MNIST の学習

### 実行準備
TensorFlow/MNISTの実行環境を用意します。

In [ ]:
# OpenHPC環境に、データやスクリプトを配置するディレクトリを作成
work_dir = 'tensorflow'
!ssh {ssh_opts} {target} mkdir -p {work_dir}

# スクリプトの転送
!scp {ssh_opts} scripts/download_mnist.py {target}:{work_dir}
!scp {ssh_opts} scripts/mnist_classify_handson.py {target}:{work_dir}
!scp {ssh_opts} scripts/mnist_classify_cnn_handson.py {target}:{work_dir}
!scp {ssh_opts} scripts/check_gpu.py {target}:{work_dir}
    
# コンテナを実行するため、ホームディレクトリのパーミッションを変更
!ssh {ssh_opts} {target} chmod 755 .

Dockerコンテナ環境からGPUを利用できることを確認します（エラーにならないことを確認）。

In [ ]:
!ssh {ssh_opts} {target} srun -N {nodes} --gpus-per-node=1 \
    docker run --gpus all --ipc=host -v /home/{user}:/home/{user} --rm tensorflow-{user} \
    python {work_dir}/check_gpu.py

### MNIST学習ジョブの実行

MNISTの学習スクリプトをSlurmのジョブとして実行します。

ジョブの実行スクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    batch_file = Path(workdir) / 'tensorflow_mnist_docker.job'
    with batch_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash
        
#SBATCH -J tensorflow-mnist-docker         # create a short name for your job
#SBATCH -o tensorflow-mnist-docker.%j.out  # Name of stdout output file (%j expands to jobId)
#SBATCH -N 1                               # Total number of nodes requested
#SBATCH -n 1                               # Total number of across all nodes
#SBATCH --gres=gpu:1                       # number of gpus per node
#SBATCH -t 00:10:00                        # Run time (hh:mm:ss)

docker run -td --gpus all -v /home/{user}:/home/{user} --rm --ipc=host --net=host --name tensorflow-{user} tensorflow-{user}
docker exec -t -u {user} -w $HOME/{work_dir} tensorflow-{user} python3 download_mnist.py
docker exec -t -u {user} -w $HOME/{work_dir} tensorflow-{user} python3 mnist_classify_handson.py
docker stop tensorflow-{user}
''')
    !cat {batch_file}
    !scp {ssh_opts} {str(batch_file)} {target}:{work_dir}

ジョブを実行します。

In [ ]:
# 以前の実行結果の削除
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && rm -rf  tensorflow-mnist*.out saved_model*'"

# ジョブの実行
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && sbatch tensorflow_mnist_docker.job'"

ジョブの実行状況を確認します。終了までに数分かかります。JOB ID 欄に上のセルで実行した JOB IDが表示されなくなれば終了です。時々本セルを`unfreeze`して再実行し、実行終了を確認してください。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することもできます。

In [ ]:
!ssh {ssh_opts} {target} squeue

ジョブの完了後に次のセルを実行してください。ジョブの出力結果が確認できます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && tail tensorflow-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1098 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0891 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0748 - accuracy: 0.9755
313/313 - 0s - loss: 0.0839 - accuracy: 0.9748

Test accuracy: 0.9747999906539917
```

学習済みモデル・重みファイルを、認識環境構築テンプレートの環境にダウンロードします。

In [ ]:
!scp {ssh_opts} -p {target}:{work_dir}/saved_model.tgz {save_dir}

以上で、学習とデータの転送は終了しました。`2.Recognition_system`を実行してください。

## 「2.Recognition_system」実行の後で

上のMNISTの学習は、[TensorFlow公式ページのチュートリアル](https://www.tensorflow.org/tutorials)の「初心者のための TensorFlow 2.0 入門」のスクリプトを基にしています。これは、フラットなニューラルネットの学習で、CNNではありません。イメージ認識ではCNNの方が適していると考えられるため、CNNによる学習スクリプトも用意しました。興味があれば、以下をお試しください。

ジョブの実行スクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    batch_file = Path(workdir) / 'tensorflow_mnist_docker.job'
    with batch_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash
        
#SBATCH -J tensorflow-mnist-docker         # create a short name for your job
#SBATCH -o tensorflow-mnist-docker.%j.out  # Name of stdout output file (%j expands to jobId)
#SBATCH -N 1                               # Total number of nodes requested
#SBATCH -n 1                               # Total number of across all nodes
#SBATCH --gres=gpu:1                       # number of gpus per node
#SBATCH -t 00:10:00                        # Run time (hh:mm:ss)

docker run -td --gpus all -v /home/{user}:/home/{user} --rm --ipc=host --net=host --name tensorflow-{user} tensorflow-{user}
docker exec -t -u {user} -w $HOME/{work_dir} tensorflow-{user} python3 download_mnist.py
docker exec -t -u {user} -w $HOME/{work_dir} tensorflow-{user} python3 mnist_classify_cnn_handson.py
docker stop tensorflow-{user}
''')
    !cat {batch_file}
    !scp {ssh_opts} {str(batch_file)} {target}:{work_dir}

ジョブを実行します。

In [ ]:
# 以前の実行結果の削除
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && rm -rf  tensorflow-mnist*.out saved_model*'"

# ジョブの実行
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && sbatch tensorflow_mnist_docker.job'"

ジョブの実行状況を確認します。終了までに数分かかります。JOB ID 欄に上のセルで実行した JOB IDが表示されなくなれば終了です。時々本セルを`unfreeze`して再実行し、実行終了を確認してください。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することもできます。

In [ ]:
!ssh {ssh_opts} {target} squeue

ジョブの完了後に次のセルを実行してください。ジョブの出力結果が確認できます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && tail tensorflow-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1098 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0891 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0748 - accuracy: 0.9755
313/313 - 0s - loss: 0.0839 - accuracy: 0.9748

Test accuracy: 0.9747999906539917
```

学習済みモデル・重みファイルを、認識環境構築テンプレートの環境にダウンロードします。

In [ ]:
!scp {ssh_opts} -p {target}:{work_dir}/saved_model2.tgz {save_dir}

## 環境の削除

本テンプレートは、OpenHPCテンプレートで構築した環境を利用しており、独自に構築した環境はありません。OpenHPC 環境の削除は、OpenHPC テンプレートの作業ディレクトリで`912-ノードの再開.ipynb`を実行してください。